### сводная статистика 

In [1]:
import pandas as pd

In [5]:
# готовим данные и даем им заголовки
df = pd.read_csv('titanic.csv', header=None)
headers = [
    'pd_id',
    'PassengerId',
    'Survived',
    'Pclass',
    'Lname',
    'Name',
    'Sex',
    'Age',
    'SibSp',
    'Parch',
    'Ticket',
    'Fare',
    'Cabin',
    'Embarked'
]
df.columns = headers


In [9]:
df.head()

,pd_id,PassengerId,Survived,Pclass,Lname,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,1,0,3,Braund,Mr. Owen Harris,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,2,1,1,Cumings,Mrs. John Bradley (Florence Briggs Thayer),female,38.0,1,0,PC 17599,71.2833,C85,C
2,2,3,1,3,Heikkinen,Miss. Laina,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,3,4,1,1,Futrelle,Mrs. Jacques Heath (Lily May Peel),female,35.0,1,0,113803,53.1000,C123,S
4,4,5,0,3,Allen,Mr. William Henry,male,35.0,0,0,373450,8.0500,NaN,S


In [10]:
# найдем средний возраст пассажирова по нашим данным
df['Age'].mean()

28.141507936507935

In [12]:
# какой средний возраст и стоимость по возрасту
# Статистика, применяемая к нескольким столбцам a DataFrame(выбор двух столбцов возвращает a DataFrame, 
# см. руководство по подмножеству данных ), вычисляется для каждого числового столбца

df[['Age', 'Fare']].median()

Age     26.0000
Fare    14.4542
dtype: float64

In [13]:
# Агрегирующая статистика может быть рассчитана для нескольких столбцов одновременно. Помните describe функцию из первого урока ?
df[['Age', 'Fare']].describe()

,Age,Fare
count,126.000000,156.000000
mean,28.141508,28.109587
std,14.613880,39.401047
min,0.830000,6.750000
25%,19.000000,8.003150
50%,26.000000,14.454200
75%,35.000000,30.371850
max,71.000000,263.000000


In [14]:
# Вместо предопределенной статистики конкретные комбинации агрегирующей статистики 
# для заданных столбцов могут быть определены с помощью DataFrame.agg() метода:
df.agg(
    {
        'Age' : ['min', 'max', 'median', 'skew'],
        'Fare' : ['min', 'max', 'median', 'mean'],
    }
)


,Age,Fare
min,0.830000,6.750000
max,71.000000,263.000000
median,26.000000,14.454200
skew,0.700367,NaN
mean,NaN,28.109587


### Агрегированная статистика, сгруппированная по категориям 

In [27]:
# попробуем посчитать агрегирвоанные данные по оперделнным полям, 
# напрммер средний возраст по полу пассажиров

df[['Sex', 'Fare']].groupby('Sex').mean()

,Fare
Sex,
female,28.460639
male,27.912998


Вычисление данной статистики (например, `mean` возраста) для каждой категории в столбце (например, мужчины/женщины в `Sex` столбце) является
распространенным шаблоном. Метод `groupby` используется для поддержки этого типа операций. В более общем плане это соответствует более общему `split-apply-combine` шаблону:

- Разделите данные на группы
- Применение функции к каждой группе независимо
- Объедините результаты в структуру данных

Шаги применения и объединения обычно выполняются вместе в `pandas`.

In [23]:
# В предыдущем примере мы сначала явно выбрали 2 столбца. 
# Если явного указания на столбы нет -> `mean`(среднего) метод применяется к каждому столбцу, содержащему числовые столбцы:

df.groupby('Sex').mean()

,pd_id,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
Sex,,,,,,,,
female,67.125,68.125,0.714286,2.464286,24.468085,0.875,0.517857,28.460639
male,83.310,84.310,0.140000,2.400000,30.326962,0.470,0.330000,27.912998


Не всегда есть смысл получать полную выборку (по всем столбам) если нужны конкретные данные по одному из них.
Например нужно посмотреть среднее значение возраста, разбив по полу. [ ] - будут поддерживаться и после указании группы агрегации

In [29]:
df.groupby('Sex')[['Age', 'Sex', 'Fare']].mean()

,Age,Fare
Sex,,
female,24.468085,28.460639
male,30.326962,27.912998


In [30]:
# Какова средняя стоимость билета для каждой из комбинаций пола и класса обслуживания?

df.groupby(['Sex', 'Pclass'])['Fare'].mean()

Sex     Pclass
female  1         90.007400
        2         21.325000
        3         15.080834
male    1         74.999600
        2         25.216200
        3         12.498633
Name: Fare, dtype: float64

### Подсчет количества записей по категориям

In [31]:
# Каково количество пассажиров в каждом из классов салона?
df['Pclass'].value_counts()

# Метод value_counts()подсчитывает количество записей для каждой категории в столбце.

3    96
1    30
2    30
Name: Pclass, dtype: int64

In [32]:
# Функция является сокращением, так как на самом деле это групповая 
# операция в сочетании с подсчетом количества записей в каждой группе:

df.groupby('Pclass')['Pclass'].count()

Pclass
1    30
2    30
3    96
Name: Pclass, dtype: int64

<font color='yellow'>ПОМНИТЕ</font>

- Статистику агрегирования можно рассчитать по целым столбцам или строкам.

- groupbyобеспечивает мощность шаблона « разделить-применить-объединить »

- value_countsэто удобный ярлык для подсчета количества записей в каждой категории переменной